**Student ID:** s225722032

**Student Name:** Phani Kiran Hemadri

**Workshop/Lab Session Time:**

In [1]:
#!pip -q install pyspark

#from google.colab import files

#uploaded = files.upload()


In [2]:
import os, glob
from google.colab import drive

print(os.getcwd())
drive.mount('/content/drive')

#!ls -lah '/content/drive/MyDrive/Colab Notebooks/business_review_submission'

file_dir='/content/drive/MyDrive/Colab Notebooks/business_review_submission'
glob.glob(os.path.join(file_dir, "*.csv"))


/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['/content/drive/MyDrive/Colab Notebooks/business_review_submission/review.csv',
 '/content/drive/MyDrive/Colab Notebooks/business_review_submission/meta-review-business.csv']

In [3]:
reviews_path = f"{file_dir}/review.csv"
meta_path    = f"{file_dir}/meta-review-business.csv"

In [4]:
# start Spark session

from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("SIT742-SparkSQL")
        .getOrCreate()
)

In [5]:
df_reviews = (
    spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv(reviews_path)
)

df_meta = (
    spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv(meta_path)
)

df_reviews.show(5, truncate=False)

df_meta.show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------+-------------+---------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+----+-------------------------------------+
|user_id                                                                                                                                            |name                                               |time         |rating                                                                                                               |te

In [6]:
# Register temp views
df_reviews.createOrReplaceTempView("reviews_raw")
df_meta.createOrReplaceTempView("meta_raw")

# Question 1.1

Using PySpark to do some data wrangling process, so that


## 1.1.1 For the none or null in text column, change it to 'no review'.

In [7]:
# confirm presence of 'text' column
df_reviews.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- text: string (nullable = true)
 |-- pics: string (nullable = true)
 |-- resp: string (nullable = true)
 |-- gmap_id: string (nullable = true)



In [8]:
# using spark sql query peek at rows that are null/empty/'none' before cleaning
spark.sql("""
SELECT *
FROM reviews_raw
WHERE text IS NULL
   OR TRIM(text) = ''
   OR LOWER(TRIM(text)) IN ('none', 'null')
""").show(truncate=True)


+--------------------+--------------------+-------------+--------------------+----+----+--------------------+--------------------+
|             user_id|                name|         time|              rating|text|pics|                resp|             gmap_id|
+--------------------+--------------------+-------------+--------------------+----+----+--------------------+--------------------+
|Wi-Fi didn't reac...|                NULL|         NULL|0x56b646ed2220b77...|NULL|NULL|                NULL|                NULL|
|1.169785198939306...|          Holly Anne|1629595392534|                   4|NULL|NULL|                NULL|0x56c8992b5dee722...|
|1.097489809617870...|       Emma Forsberg|1629598102068|                   5|NULL|NULL|                NULL|0x56c8992b5dee722...|
|1.069279219833724...|       sherry miller|1630540491895|                   5|NULL|NULL|                NULL|0x56c8992b5dee722...|
|1.176554580456088...|            Amy Lieb|1630292802235|                   4|NULL|

In [9]:
# replace None/null/""/"none" in text with "no review"
# create new temp view 'review_clean' adding new column 'text_clean'

spark.sql("""
CREATE OR REPLACE TEMP VIEW reviews_clean AS
SELECT
  *,
  CASE
    WHEN text IS NULL
      OR TRIM(text) = ''
      OR LOWER(TRIM(text)) IN ('none', 'null')
    THEN 'no review'
    ELSE text
  END AS text_clean
FROM reviews_raw
""")




DataFrame[]

In [10]:
# peek few rows after cleaning null/none 'text' column with 'no review'
spark.sql("""
SELECT user_id, name, text AS original_text, text_clean
FROM reviews_clean
""").show(truncate=True)

+--------------------+--------------------+--------------------+--------------------+
|             user_id|                name|       original_text|          text_clean|
+--------------------+--------------------+--------------------+--------------------+
|1.091298048426862...|          Nicki Gore|We always stay he...|We always stay he...|
|1.132409264057589...|       Allen Ratliff|Great campground ...|Great campground ...|
|1.130448378911412...|   Jonathan Tringali|We tent camped he...|We tent camped he...|
|There is a bath h...| 2 restrooms (sin...| but they lack ve...| but they lack ve...|
|Wi-Fi didn't reac...|                NULL|                NULL|           no review|
|1.103291551475920...|              S Blad|This place is jus...|This place is jus...|
|1.08989634908602e+20|      Daniel Formoso|Probably the nice...|Probably the nice...|
|1.157388655487960...|       Victoria Cook|Great, slept like...|Great, slept like...|
|1.163932319879565...|              Matt H|It is alway

In [11]:
# print count of replaced items
spark.sql("""
SELECT COUNT(*) AS replaced_count
FROM reviews_clean
WHERE text_clean = 'no review'
""").show()

+--------------+
|replaced_count|
+--------------+
|        249931|
+--------------+



## 1.1.2 Process the content in time column, and convert the strings from time to yyyy-mm-dd format in the new column as newtime and show the first 5 rows.

In [12]:
# first lets look at schema so we see column names and types
spark.sql("DESCRIBE TABLE reviews_clean").show(truncate=False)

+----------+---------+-------+
|col_name  |data_type|comment|
+----------+---------+-------+
|user_id   |string   |NULL   |
|name      |string   |NULL   |
|time      |string   |NULL   |
|rating    |string   |NULL   |
|text      |string   |NULL   |
|pics      |string   |NULL   |
|resp      |string   |NULL   |
|gmap_id   |string   |NULL   |
|text_clean|string   |NULL   |
+----------+---------+-------+



In [13]:
# lets review the lengths of 'time' strings
# In general milli seconds epoch string will be of length 13
# other lengths suggest improper data

spark.sql("""
SELECT
  LENGTH(time) AS len,
  COUNT(*)     AS cnt
FROM reviews_clean
GROUP BY LENGTH(time)
ORDER BY cnt DESC
""").show()

+----+------+
| len|   cnt|
+----+------+
|  13|521573|
|NULL| 24815|
| 112|   138|
| 224|    80|
|  40|    74|
|  42|    70|
|  20|    69|
|  14|    68|
|  18|    68|
|   6|    66|
|   7|    66|
|  21|    64|
|   5|    62|
|  41|    62|
|  11|    62|
|  44|    61|
| 336|    61|
|  39|    60|
|  23|    60|
|  29|    59|
+----+------+
only showing top 20 rows



In [14]:
# lets create a view that adds a column 'newtime' formatted as yyyy-MM-dd
spark.sql("""
CREATE OR REPLACE TEMP VIEW reviews_with_newtime AS
SELECT
  t.*,
  date_format(
    to_timestamp(
      from_unixtime( CAST(CAST(t.time AS BIGINT) / 1000 AS BIGINT) )
    ),
    'yyyy-MM-dd'
  ) AS newtime
FROM reviews_clean t
""")

DataFrame[]

In [15]:
# lets view original 'time' vs 'newtime' in yyyy-MM-dd format
spark.sql("""
SELECT user_id, name, time, newtime
FROM reviews_with_newtime
""").show(truncate=True)

# count rows where parsing failed and newtime is NULL
spark.sql("""
SELECT COUNT(*) AS null_newtime_count
FROM reviews_with_newtime
WHERE newtime IS NULL
""").show()

+--------------------+--------------------+-------------+----------+
|             user_id|                name|         time|   newtime|
+--------------------+--------------------+-------------+----------+
|1.091298048426862...|          Nicki Gore|1566331951619|2019-08-20|
|1.132409264057589...|       Allen Ratliff|1504917982385|2017-09-09|
|1.130448378911412...|   Jonathan Tringali|1474765901185|2016-09-25|
|There is a bath h...| 2 restrooms (sin...|       toilet|      NULL|
|Wi-Fi didn't reac...|                NULL|         NULL|      NULL|
|1.103291551475920...|              S Blad|1472858535682|2016-09-02|
|1.08989634908602e+20|      Daniel Formoso|1529649811341|2018-06-22|
|1.157388655487960...|       Victoria Cook|1466170294782|2016-06-17|
|1.163932319879565...|              Matt H|1625369270215|2021-07-04|
|1.179859920593560...|             Claudie|1629350418882|2021-08-19|
|1.156582466444838...|       Crystal Dahle|1630382185833|2021-08-31|
|1.036557557423225...|           J